In [1]:
# load the libraries you'll need
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [13]:
# Save your html file with this function that way you don't male repeated requests to a website
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

# Open it back up with this function
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

In [20]:
# This is the prefix of the website for later use
prefix = "http://www.ifar.org/"

In [9]:
url = 'http://www.ifar.org/catalogues_raisonnes.php?alpha=&searchtype=artist&published=1&inPrep=1&artist=&author='
path = 'C:/Users/ramon/OneDrive/INSEAD/auction_proj'

In [10]:
# create a conection with the website
r = requests.get(url)
print(r.content[:100])

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtm'


In [11]:
# assign the content of the website to a new variable using the method .content then save to not make more requests
save_html(r.content, 'auctions_html')

In [14]:
# Load back in the hmtl of the page
html = open_html('auctions_html')

In [36]:
# instantiate a BeatifulSoup object with the content and the type of parser you'll use
soup = BeautifulSoup(html, "html.parser")

In [37]:
# Inspect soup to make sure data was loaded successfully
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html>
<head>
<title>International Foundation for Art Research (IFAR)-Catalogues Raisonnés</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="International Foundation for Art Research - Catalogues raisonnés — scholarly compilations of an artist's body of work — are critical tools for researching the provenance and attribution of artwork. Here are two new electronic resources — a database of published catalogues raisonnés and one of catalogues in preparation. They can be searched separately or together and are updated regularly." name="description"/>
<meta content="Art, Stolen Art, Art Research, Provenance, Art Law, Catalogues Raisonnes" name="keywords"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="noarchive" name="googlebot"/>
<meta content="0" http-equi

In [70]:
# Select the rows of the tables that contain the data
rows = soup.select('tr')

In [77]:
#Inspect a row
row = rows[1]

In [107]:
name = row.select_one('.g_center_name').get_text('|').strip()

In [108]:
print(name)

Aachen, Hans von|click to learn more


In [93]:
allsides_page = row.select_one('.g_center_name a')['href']
allsides_page = prefix + allsides_page

print(allsides_page)

http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=
